In [3]:
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.data import Data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from dsc2024 import datasets


In [4]:
df = datasets.get_train_dataset()
df_features = df.select_dtypes(include='number').drop("espera", axis=1)
df_features.fillna(0, inplace=True)
df_features.info()

2024-10-23 09:41:26.672 | WARNING  | dsc2024.handling:parse_metar_as_dataframe:57 - Error(row=149631): <class 'metpy.io._metar_parser.metar_parser.ParseError'> - Line 1: expected one of:

    - [\d] from METAR::datetime
    - "Z" from METAR::datetime

     1 | METAR COR SBSV 051300 10012KT 9999 FEW021 SCT050 28/24 Q1014=
                              ^
2024-10-23 09:41:26.677 | WARNING  | dsc2024.handling:parse_metar_as_dataframe:57 - Error(row=149639): <class 'metpy.io._metar_parser.metar_parser.ParseError'> - Line 1: expected one of:

    - [\d] from METAR::datetime
    - "Z" from METAR::datetime

     1 | METAR COR SBSV 051300 10012KT 9999 FEW021 SCT050 28/24 Q1014=
                              ^
2024-10-23 09:41:26.687 | WARNING  | dsc2024.handling:parse_metar_as_dataframe:57 - Error(row=149657): <class 'metpy.io._metar_parser.metar_parser.ParseError'> - Line 1: expected one of:

    - [\d] from METAR::datetime
    - "Z" from METAR::datetime

     1 | METAR COR SBSV 051400 09013KT

<class 'pandas.core.frame.DataFrame'>
Index: 211679 entries, 504a62621cd231d6ab67e674ce538cd3 to c962a2267ae4fe0afa4c3542ebdbd403
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   prev_troca_cabeceira           211679 non-null  int64  
 1   troca_cabeceira_hora_anterior  211679 non-null  int64  
 2   destino_altitude               211679 non-null  int64  
 3   destino_comprimento            211679 non-null  int64  
 4   destino_largura                211679 non-null  int64  
 5   metar_latitude                 211679 non-null  float64
 6   metar_longitude                211679 non-null  float64
 7   metar_elevation                211679 non-null  float64
 8   metar_wind_direction           211679 non-null  float64
 9   metar_wind_speed               211679 non-null  float64
 10  metar_wind_gust                211679 non-null  float64
 11  metar_visibility               211679 n

In [5]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, BatchNorm
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Set a random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False     # Disable CuDNN benchmark to ensure determinism

# Call the seed function at the start of your script
set_seed(42)  # You can use any integer as the seed value


In [6]:
NUM_EDGE_FEATURES = len(df_features.columns)

# GNN model with a single GATConv layer for binary classification
class FlightGNNWithGAT(torch.nn.Module):
    def __init__(self, num_features=1, hidden_size=32, target_size=1):
        super(FlightGNNWithGAT, self).__init__()
        self.hidden_size = hidden_size
        self.num_features = num_features
        self.target_size = target_size

        # Single GATConv layer
        self.conv = GATConv(self.num_features, self.hidden_size, edge_dim=NUM_EDGE_FEATURES)

        # Linear layer for final edge-level classification output
        self.edge_mlp = nn.Linear(2 * self.hidden_size + NUM_EDGE_FEATURES, self.target_size)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # Apply GATConv layer
        x = self.conv(x, edge_index, edge_attr=edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        # Concatenate node embeddings of the source and destination nodes of each edge, along with edge attributes
        row, col = edge_index
        edge_rep = torch.cat([x[row], x[col], edge_attr], dim=1)

        return torch.sigmoid(self.edge_mlp(edge_rep))

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # Probabilities
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

def train_test_split_data(data, test_size=0.2):
    num_edges = data.edge_index.shape[1]
    edge_indices = list(range(num_edges))

    train_indices, test_indices = train_test_split(edge_indices, test_size=test_size, random_state=42)

    train_mask = torch.zeros(num_edges, dtype=torch.bool)
    test_mask = torch.zeros(num_edges, dtype=torch.bool)

    train_mask[train_indices] = True
    test_mask[test_indices] = True

    return train_mask, test_mask

def train(model, data, train_mask, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)  # Edge-level output
    loss = criterion(out[train_mask], data.y[train_mask])

    loss.backward()
    optimizer.step()

    return loss.item()

def test(model, data, test_mask):
    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = (out[test_mask] > 0.5).float()

        y_true = data.y[test_mask].numpy()
        y_pred = preds.numpy()

        acc = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred)

        return acc, precision, recall, f1, cm

def prepare_data(df):
    # Convert your DataFrame into PyG Data format
    node_set = set(df['origem']).union(set(df['destino']))
    node_to_idx = {node: i for i, node in enumerate(node_set)}

    edge_index = torch.tensor([[node_to_idx[o], node_to_idx[d]] for o, d in zip(df['origem'], df['destino'])], dtype=torch.long).t().contiguous()

    # Edge features
    edge_attr = torch.tensor(df_features.values, dtype=torch.float)

    # Target edge feature to predict ('espera')
    y = torch.tensor(df['espera'].values, dtype=torch.float).unsqueeze(1)

    # Dummy node features (e.g., all ones)
    num_nodes = len(node_to_idx)
    node_features = torch.ones((num_nodes, 1))  # Dummy node features (1 for each node)

    # Create Data object
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=y)

    return data

# Prepare the data
data = prepare_data(df)

# Split the data into train and test masks
train_mask, test_mask = train_test_split_data(data, test_size=0.2)

# Initialize the model
model = FlightGNNWithGAT(num_features=1, hidden_size=32, target_size=1)
criterion = FocalLoss(alpha=0.1, gamma=70)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 100
train_losses = []
for epoch in range(epochs):
    train_loss = train(model, data, train_mask, optimizer, criterion)
    train_losses.append(train_loss)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')

# Testing
acc, precision, recall, f1, cm = test(model, data, test_mask)
print(f'Test Accuracy: {acc:.4f}')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')
print(f'Confusion Matrix:\n{cm}')


Epoch 0, Loss: 0.0000
Epoch 10, Loss: 0.0000
Epoch 20, Loss: 0.0000
Epoch 30, Loss: 0.0000
Epoch 40, Loss: 0.0000
Epoch 50, Loss: 0.0000
Epoch 60, Loss: 0.0000
Epoch 70, Loss: 0.0000
Epoch 80, Loss: 0.0000
Epoch 90, Loss: 0.0000
Test Accuracy: 0.9483
Precision: 0.0288, Recall: 0.0625, F1-Score: 0.0395
Confusion Matrix:
[[40101  1515]
 [  675    45]]


In [13]:

NUM_EDGE_FEATURES = len(df_features.columns)

class FlightGNNWithGAT(torch.nn.Module):
    def __init__(self, num_features=1, hidden_size=32, num_layers=10, target_size=1):
        super(FlightGNNWithGAT, self).__init__()
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        self.batch_norms = torch.nn.ModuleList()  # For BatchNorm layers
        
        # First GATConv layer
        self.convs.append(GATConv(num_features, hidden_size, edge_dim=NUM_EDGE_FEATURES))
        self.batch_norms.append(BatchNorm(hidden_size))  # BatchNorm for the first layer
        
        # Hidden GATConv layers
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_size, hidden_size, edge_dim=NUM_EDGE_FEATURES))
            self.batch_norms.append(BatchNorm(hidden_size))  # BatchNorm for hidden layers

        # Linear layers for edge representation refinement
        self.fc1 = nn.Linear(2 * hidden_size + NUM_EDGE_FEATURES, hidden_size)  # First linear layer
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second linear layer
        self.edge_mlp = nn.Linear(hidden_size, target_size)  # Final linear layer for edge-level classification

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # Apply GATConv layers with ReLU, BatchNorm, and dropout
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index, edge_attr=edge_attr)
            x = self.batch_norms[i](x)  # Apply Batch Normalization
            x = F.relu(x)
            x = F.dropout(x, training=self.training)

        # Concatenate node embeddings of the source and destination nodes of each edge, along with edge attributes
        row, col = edge_index
        edge_rep = torch.cat([x[row], x[col], edge_attr], dim=1)

        # Pass through linear layers
        edge_rep = F.relu(self.fc1(edge_rep))  # First linear layer
        edge_rep = F.dropout(edge_rep, training=self.training)  # Dropout for regularization
        edge_rep = F.relu(self.fc2(edge_rep))  # Second linear layer

        return torch.sigmoid(self.edge_mlp(edge_rep))



class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # Probabilities
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        return F_loss.mean()


def train_test_split_data(data, test_size=0.2):
    num_edges = data.edge_index.shape[1]
    edge_indices = list(range(num_edges))

    train_indices, test_indices = train_test_split(edge_indices, test_size=test_size)

    train_mask = torch.zeros(num_edges, dtype=torch.bool)
    test_mask = torch.zeros(num_edges, dtype=torch.bool)

    train_mask[train_indices] = True
    test_mask[test_indices] = True

    return train_mask, test_mask


def train(model, data, train_mask, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    print("DEBUG: ", data)
    out = model(data)  # Edge-level output
    loss = criterion(out[train_mask], data.y[train_mask])

    loss.backward()
    optimizer.step()

    return loss.item()


def test(model, data, test_mask):
    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = (out[test_mask] > 0.5).float()

        y_true = data.y[test_mask].numpy()
        y_pred = preds.numpy()

        acc = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred)

        return acc, precision, recall, f1, cm


def prepare_data(df):
    # Convert your DataFrame into PyG Data format
    node_set = set(df['origem']).union(set(df['destino']))
    node_to_idx = {node: i for i, node in enumerate(node_set)}

    edge_index = torch.tensor([[node_to_idx[o], node_to_idx[d]] for o, d in zip(df['origem'], df['destino'])], dtype=torch.long).t().contiguous()

    # Edge features
    edge_attr = torch.tensor(df_features.values, dtype=torch.float)

    # Target edge feature to predict ('espera')
    y = torch.tensor(df['espera'].values, dtype=torch.float).unsqueeze(1)

    # Dummy node features (e.g., all ones)
    num_nodes = len(node_to_idx)
    node_features = torch.ones((num_nodes, 1))  # Dummy node features (1 for each node)

    # Create Data object
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=y)

    return data


# Prepare the data
data = prepare_data(df)

# Split the data into train and test masks
train_mask, test_mask = train_test_split_data(data, test_size=0.2)

# Initialize the model
model = FlightGNNWithGAT(num_features=1, hidden_size=32, target_size=1)

# Initialize the loss function and optimizer
criterion = FocalLoss(alpha=0.1, gamma=75)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 100
train_losses = []
for epoch in range(epochs):
    train_loss = train(model, data, train_mask, optimizer, criterion)
    train_losses.append(train_loss)
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {train_loss:.4f}')

# Testing
acc, precision, recall, f1, cm = test(model, data, test_mask)
print(f'Test Accuracy: {acc:.4f}')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')
print(f'Confusion Matrix:\n{cm}')


DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
Epoch 0, Loss: 0.0000
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
DEBUG:  Data(x=[12, 1], edge_index=[2, 211679], edge_attr=[211679, 41], y=[211679, 1])
Epoch 10, Loss: 0.000